In [1]:
import pandas as pd
import sqlite3

dataframes needed:
- one with hospitals (Michael said there wasn't any real programatic way to do it, get a df with entity_type = 2 and then figure out the real list by looking at names)
- use hospital df to get list of referees

In [32]:
# building table for hospitals
# cbsa = 34980, either hardcode or use a cte to get the value where usps_zip_pref_city = "NASHVILLE" and usps_zip_pref_state = "TN" (filter based on referee cbsa)
query = """
SELECT npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE entity_type_code = 2
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp_sqlite = pd.read_sql(query, db)

hosp_sqlite.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1881697092,2,"RHS, INC.",1330 TROTWOOD AVE,None,COLUMBIA,TN,38401,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
1,1326041534,2,MURFREESBORO CONVALESCENT SERVICE,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,37130,341600000X,Transportation Services,Ambulance,Ambulance
2,1063415834,2,ADVANCED DIABETIC SERVICES,151 HERITAGE PARK DR,STE 403,MURFREESBORO,TN,37129,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
3,1538162334,2,TENNESSEE IMAGING ALLIANCE LLC,2628 N MOUNT JULIET RD,None,MOUNT JULIET,TN,37122,261QR0208X,Ambulatory Health Care Facilities,Clinic/Center,Mobile Radiology Clinic/Center
4,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center


In [122]:
hosp_sqlite['code_group'].value_counts()

Allopathic & Osteopathic Physicians                                             2002
Ambulatory Health Care Facilities                                               1408
Suppliers                                                                       1354
Agencies                                                                         888
Dental Providers                                                                 662
Behavioral Health & Social Service Providers                                     440
Nursing & Custodial Care Facilities                                              332
Chiropractic Providers                                                           332
Physician Assistants & Advanced Practice Nursing Providers                       329
Transportation Services                                                          327
Other Service Providers                                                          317
Respiratory, Developmental, Rehabilitative and Restorative Servic

In [167]:
query = """
SELECT *
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
WHERE provider_organization_name_legal_business_name LIKE 'NASHVILLE GENERAL%';
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp_find = pd.read_sql(query, db)

hosp_find.head()

,npi,entity_type_code,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,extracted_taxonomy_code,code,grouping,classification,display_name
0,1932141504,2,NASHVILLE GENERAL HOSPITAL,None,None,None,None,None,None,1818 ALBION ST,None,NASHVILLE,TN,37208,3336I0012X,3336I0012X,Suppliers,Pharmacy,Institutional Pharmacy
1,1568551455,2,NASHVILLE GENERAL HOSPITAL,None,None,None,None,None,None,1818 ALBION ST,ATTN: MARTHA LAMPLEY,NASHVILLE,TN,37208,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,General Acute Care Hospital
2,1952673253,2,NASHVILLE GENERAL RADIOLOGICAL SERVICES,None,None,None,None,None,None,1818 ALBION ST,None,NASHVILLE,TN,37208,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology Physician
3,1881132728,2,NASHVILLE GENERAL HOSPITAL,None,None,None,None,None,None,515 MADISON ST,APT.424,NASHVILLE,TN,37208,261QP2300X,261QP2300X,Ambulatory Health Care Facilities,Clinic/Center,Primary Care Clinic/Center


In [170]:
# pick major hospitals in Nashville
hosp_sqlite[hosp_sqlite['code_group'].isin(['Hospital', 'Hospitals', 'Hospital Units'])]['provider_org_name'].unique()

# besides the obvious
# Vanderbilt - Shelbyville
# Ascension St Thomas - River Park, Cannon, Smith, Rutherford, Trousdale, Sumner, Baptist, Seton, Riverview, Smith, Trousdale
# HCA Tristar - HTI, Central, Hendersonville, Northcrest, Centennial
# Nashville General - Meharry
hosp_list = ["BAPTIST WOMEN'S HEALTH CENTER LLC",
       'SHELBYVILLE HOSPITAL COMPANY LLC',
       'HCA HEALTH SERVICES OF TENNESSEE, INC.',
       'VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP',
       'VANDERBILT MEDICAL CENTER',
       'RIVER PARK HOSPITAL LLC',
       'HTI MEMORIAL HOSPITAL CORPORATION',
       'CENTRAL TENNESSEE HOSPITAL CORPORATION',
       'CANNON COUNTY HOSPITAL LLC',
       'HENDERSONVILLE HOSPITAL CORPORATION',
       'SAINT THOMAS WEST HOSPITAL',
       'HCA-HEALTHONE LLC',
       'VANDERBILT UNIVERSITY MEDICAL CENTER',
       'CANNONB COUNTY HOSPITAL, LLC',
       'SMITH COUNTY MEMORIAL HOSPITAL, LLC',
       'FRANK T RUTHERFORD MEMORIAL HOSPITAL, INC',
       'TROUSDALE MEDICAL CENTER, INC.',
       'SUMNER REGIONAL HEALTH SYSTEMS, INC',
       'BAPTIST HOSPITAL SYSTEMS INC',
       'CANNON COUNTY HOSPITAL, LLC',
       'VANDERBILT UNIV. MEDICAL CENTER',
       'NORTHCREST MEDICAL CENTER',
       'NASHVILLE GENERAL HOSPITAL',
       'VANDERBILT COMMUNITY MENTAL HEALTH CENTER',
       'SAINT THOMAS RUTHERFORD HOSPITAL',
       'SETON CORPORATION',
       'FRANK T RUTHERFORD HOSPITAL INC',
       "VANDERBILT CHILDREN'S",
       'VANDERBILT CHILDRENS HOSPITAL',
       'RIVER PARK HOSPITAL INC',
       'SUMNER REGIONAL HEALTH SYSTEM INC',
       "MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL",
       'VANDERBILT PSYCHIATRIC HOSPITAL, LLC',
       'VANDERBILT HEART AND VASCULAR INSTITUTE',
       'VANDERBILT ORTHOPAEDIC INSTITUTE',
       'ST. THOMAS NP, LLC',
       'SUMNER REGIONAL MEDICAL CENTER LLC',
       'RIVERVIEW MEDICAL CENTER LLC',
       'SMITH COUNTY MEMORIAL HOSPITAL LLC',
       'TROUSDALE MEDICAL CENTER LLC',
       'MEHARRY MEDICAL COLLEGE',
       'SAINT THOMAS HEART',
       'RIVERVIEW MEDICAL CENTER, LLC',
       'CENTENNIAL',
       "MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDERBILT",
       'VANDERBILT UNIVERSITY HOSPITAL',
       'SAINT THOMAS RIVER PARK HOSPITAL, LLC',
       'SAINT THOMAS STONES RIVER HOSPITAL, LLC',
       'SAINT THOMAS DEKALB HOSPITAL, LLC',
       'VANDERBILT UNIVERSITY',
       'VANDERBILT PSYCHIATRIC HOSPITAL',
       'TRISTAR MAURY BEHAVIORAL HEALTHCARE, LLC',
       'VANDERBILT BEDFORD HOSPITAL, LLC',
       'SAINT THOMAS REHABILITATION HOSPITAL, LLC']

In [171]:
# list of hospital NPIs to build profile
# all located in Nashville CBSA
hosp_npi = tuple(hosp_sqlite[hosp_sqlite['provider_org_name'].isin(hosp_list)]['npi'].tolist())

In [172]:
# get dataframe of hospitals in Nashville CBSA using previously generated NPI list
query = f"""
SELECT CAST(npi AS int64) AS npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE npi IN {hosp_npi}
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp = pd.read_sql(query, db)

hosp.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1003211145,2,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,2200 CHILDRENS WAY PRESTON RESEARCH BLDG 395,None,NASHVILLE,TN,37232,282NC2000X,Hospitals,General Acute Care Hospital,Children's Hospital
1,1013215086,2,VANDERBILT UNIVERSITY,1161 21ST AVE S RM 1815,THE VANDERBILT CLINIC,NASHVILLE,TN,37232,3336C0003X,Suppliers,Pharmacy,Community/Retail Pharmacy
2,1013435908,2,SHELBYVILLE HOSPITAL COMPANY LLC,3335 HIGHWAY 41A N,None,UNIONVILLE,TN,37180,261QR1300X,Ambulatory Health Care Facilities,Clinic/Center,Rural Health Clinic/Center
3,1013553155,2,VANDERBILT UNIVERSITY MEDICAL CENTER,1211 MEDICAL CENTER DR # 2906,None,NASHVILLE,TN,37232,333600000X,Suppliers,Pharmacy,Pharmacy
4,1013932946,2,VANDERBILT UNIVERSITY,2611 WEST END AVENUE,SUITE 280,NASHVILLE,TN,37203,367A00000X,Physician Assistants & Advanced Practice Nursi...,Advanced Practice Midwife,Advanced Practice Midwife


In [173]:
# query to get to_npi from hop data where to_npi is in hospital list
query = f"""
SELECT from_npi,
    to_npi,
    patient_count,
    transaction_count,
    average_day_wait,
    std_day_wait
FROM hop
WHERE to_npi IN {hosp_npi};
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hop_to_hosp = pd.read_sql(query, db)

hop_to_hosp.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1467604470,1013435908,114,199,1.417,14.418
1,1932170750,1013435908,90,122,28.459,52.010
2,1518115062,1023055126,36,56,14.321,18.480
3,1518014323,1023055126,397,791,0.039,0.246
4,1508961996,1023055126,47,52,40.808,61.501


to construct dataframe for provider profile:
- query to get rest of provider info (from_npi in hop_to_hosp)
- inner join on npi = from_npi in hop_to_hosp
- join hop_to_hosp with hosp on npi = to_npi to get hospital names (think I can drop most columns from hospital, fine to go back and keep some later if needed)

In [174]:
# tuple of providers who referred to area hospitals
provider_npi = tuple(hop_to_hosp['from_npi'].unique().tolist())

In [175]:
# get dataframe of hospitals in Nashville CBSA using previously generated NPI list
query = f"""
SELECT CAST(npi AS int64) AS npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_last_name_legal_name AS provider_last_name,
    provider_first_name,
    provider_middle_name,
    provider_name_prefix_text AS provider_name_prefix,
    provider_name_suffix_text AS provider_name_suffix,
    provider_credential_text AS provider_credential,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE npi IN {provider_npi};
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    provider = pd.read_sql(query, db)

provider.head()

,npi,entity_type,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1003013160,1,None,GRABENSTEIN,WILLIAM,P.,DR.,None,M.D.,1822 MEMORIAL DR,None,CLARKSVILLE,TN,37043,207Q00000X,Allopathic & Osteopathic Physicians,Family Medicine,Family Medicine Physician
1,1003013160,1,None,GRABENSTEIN,WILLIAM,P.,DR.,None,M.D.,1822 MEMORIAL DR,None,CLARKSVILLE,TN,37043,207Q00000X,Allopathic & Osteopathic Physicians,Family Medicine,Family Medicine Physician
2,1003013947,1,None,OTTO,ROBERT,JOEL,DR.,None,M.D.,2400 PATTERSON ST STE 100,None,NASHVILLE,TN,37203,207X00000X,Allopathic & Osteopathic Physicians,Orthopaedic Surgery,Orthopaedic Surgery Physician
3,1003019902,1,None,ACOSTA,LEALANI,MAE,None,None,MD,3601 TVC,None,NASHVILLE,TN,37232,2084N0400X,Allopathic & Osteopathic Physicians,Psychiatry & Neurology,Neurology Physician
4,1003031261,1,None,RUBRIGHT,JAMES,HARDMAN,None,None,M.D.,345 23RD AVE N,SUITE 212,NASHVILLE,TN,37203,207XS0106X,Allopathic & Osteopathic Physicians,Orthopaedic Surgery,Orthopaedic Hand Surgery Physician


In [176]:
provider.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   npi                   9600 non-null   int64 
 1   entity_type           9531 non-null   object
 2   provider_org_name     2700 non-null   object
 3   provider_last_name    6831 non-null   object
 4   provider_first_name   6831 non-null   object
 5   provider_middle_name  5182 non-null   object
 6   provider_name_prefix  2823 non-null   object
 7   provider_name_suffix  321 non-null    object
 8   provider_credential   6630 non-null   object
 9   provider_address_1    9531 non-null   object
 10  provider_address_2    2633 non-null   object
 11  provider_city         9531 non-null   object
 12  provider_state        9531 non-null   object
 13  provider_post_code    9531 non-null   object
 14  tax_code              9505 non-null   object
 15  code_group            9505 non-null   

In [177]:
# rmerge hop data with provider
referral = pd.merge(provider, hop_to_hosp, left_on = 'npi', right_on = 'from_npi', how = 'inner').drop(columns = 'from_npi')

In [178]:
# add hospital name and code_class
referral = pd.merge(referral, hosp[['npi', 'provider_org_name', 'code_classification']], how = 'inner', left_on = 'to_npi', right_on = 'npi').drop(columns = 'npi_y').rename({'npi_x': 'provider_npi', 'provider_org_name_x': 'provider_org_name', 'code_classification_x': 'provider_code_class', 'provider_org_name_y': 'hospital_name', 'code_classification_y': 'hospital_code_class'}, axis = 1)

In [179]:
# clean up hospital names
# besides the obvious
# Vanderbilt - Shelbyville
# Ascension St Thomas - River Park, Cannon, Smith, Rutherford, Trousdale, Sumner, Baptist, Seton, Riverview, Smith, Trousdale
# HCA Tristar - HTI, Central, Hendersonville, Northcrest, Centennial
# Nashville General - Meharry
name_replace = {"BAPTIST WOMEN'S HEALTH CENTER LLC" : 'Ascension St Thomas',
       'SHELBYVILLE HOSPITAL COMPANY LLC' : 'Vanderbilt',
       'HCA HEALTH SERVICES OF TENNESSEE, INC.' : 'HCA Tristar',
       'VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP' : 'Vanderbilt',
       'VANDERBILT MEDICAL CENTER' : 'Vanderbilt',
       'RIVER PARK HOSPITAL LLC' : 'Ascension St Thomas',
       'HTI MEMORIAL HOSPITAL CORPORATION' : 'HCA Tristar',
       'CENTRAL TENNESSEE HOSPITAL CORPORATION' : 'HCA Tristar',
       'CANNON COUNTY HOSPITAL LLC' : 'Ascension St Thomas',
       'HENDERSONVILLE HOSPITAL CORPORATION' : 'HCA Tristar',
       'SAINT THOMAS WEST HOSPITAL' : 'Ascension St Thomas',
       'HCA-HEALTHONE LLC' : 'HCA Tristar',
       'VANDERBILT UNIVERSITY MEDICAL CENTER' : 'Vanderbilt',
       'CANNONB COUNTY HOSPITAL, LLC' : 'Ascension St Thomas',
       'SMITH COUNTY MEMORIAL HOSPITAL, LLC' : 'Ascension St Thomas',
       'FRANK T RUTHERFORD MEMORIAL HOSPITAL, INC' : 'Ascension St Thomas',
       'TROUSDALE MEDICAL CENTER, INC.' : 'Ascension St Thomas',
       'SUMNER REGIONAL HEALTH SYSTEMS, INC' : 'Ascension St Thomas',
       'BAPTIST HOSPITAL SYSTEMS INC' : 'Ascension St Thomas',
       'CANNON COUNTY HOSPITAL, LLC' : 'Ascension St Thomas',
       'VANDERBILT UNIV. MEDICAL CENTER' : 'Vanderbilt',
       'NORTHCREST MEDICAL CENTER' : 'HCA Tristar',
       'NASHVILLE GENERAL HOSPITAL' : 'Nashville General',
       'VANDERBILT COMMUNITY MENTAL HEALTH CENTER' : 'Vanderbilt',
       'SAINT THOMAS RUTHERFORD HOSPITAL' : 'Ascension St Thomas',
       'SETON CORPORATION' : 'Ascension St Thomas',
       'FRANK T RUTHERFORD HOSPITAL INC' : 'Ascension St Thomas',
       "VANDERBILT CHILDREN'S" : 'Vanderbilt',
       'VANDERBILT CHILDRENS HOSPITAL' : 'Vanderbilt',
       'RIVER PARK HOSPITAL INC' : 'Ascension St Thomas',
       'SUMNER REGIONAL HEALTH SYSTEM INC' : 'Ascension St Thomas',
       "MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL" : 'Vanderbilt',
       'VANDERBILT PSYCHIATRIC HOSPITAL, LLC' : 'Vanderbilt',
       'VANDERBILT HEART AND VASCULAR INSTITUTE' : 'Vanderbilt',
       'VANDERBILT ORTHOPAEDIC INSTITUTE' : 'Vanderbilt',
       'ST. THOMAS NP, LLC' : 'Ascension St Thomas',
       'SUMNER REGIONAL MEDICAL CENTER LLC' : 'Ascension St Thomas',
       'RIVERVIEW MEDICAL CENTER LLC' : 'Ascension St Thomas',
       'SMITH COUNTY MEMORIAL HOSPITAL LLC' : 'Ascension St Thomas',
       'TROUSDALE MEDICAL CENTER LLC' : 'Ascension St Thomas',
       'MEHARRY MEDICAL COLLEGE' : 'Nashville General',
       'SAINT THOMAS HEART' : 'Ascension St Thomas',
       'RIVERVIEW MEDICAL CENTER, LLC' : 'Ascension St Thomas',
       'CENTENNIAL' : 'HCA Tristar',
       "MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDERBILT" : 'Vanderbilt',
       'VANDERBILT UNIVERSITY HOSPITAL' : 'Vanderbilt',
       'SAINT THOMAS RIVER PARK HOSPITAL, LLC' : 'Ascension St Thomas',
       'SAINT THOMAS STONES RIVER HOSPITAL, LLC' : 'Ascension St Thomas',
       'SAINT THOMAS DEKALB HOSPITAL, LLC' : 'Ascension St Thomas',
       'VANDERBILT UNIVERSITY' : 'Vanderbilt',
       'VANDERBILT PSYCHIATRIC HOSPITAL' : 'Vanderbilt',
       'TRISTAR MAURY BEHAVIORAL HEALTHCARE, LLC' : 'HCA Tristar',
       'VANDERBILT BEDFORD HOSPITAL, LLC' : 'Vanderbilt',
       'SAINT THOMAS REHABILITATION HOSPITAL, LLC' : 'Ascension St Thomas'}

In [180]:
referral['hospital_name'] = referral['hospital_name'].replace(name_replace)

In [181]:
referral['hospital_name'].value_counts()

Vanderbilt             11642
HCA Tristar             4833
Ascension St Thomas     4818
Nashville General        190
Name: hospital_name, dtype: int64

In [182]:
referral.to_csv('../data/referral.csv', index = False)